<a href="https://colab.research.google.com/github/tomheston/A-Comparison-of-the-Relative-Risk-Index-with-Unit-Fragilty/blob/main/Generate_Significant_Tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#
#
# This program calculates the fragility index, the unit fragility index,
# the relative risk index, and the p-value for a standard 2x2 contingency
# table were the base p-value is < 0.05
#
# Thomas F. Heston
# GNU GPL v3.0
# Please provide a citation to this program or the associated
# manuscript if you find this useful. Thank you!
# https://faculty.washington.edu/theston
#
#

import pandas as pd
from scipy.stats import fisher_exact
import numpy as np

lowran = 1
highestran = 100
toppv = 0.05
lowpv = 0.000999
cases_per_range = 200
pv_ranges = [(0.000999, 0.005), (0.005, 0.01), (0.01, 0.015), (0.015, 0.02), (0.02, 0.025), (0.025, 0.03), (0.03, 0.035), (0.035, 0.04), (0.04, 0.045), (0.045, 0.05)]
total_cases = len(pv_ranges) * cases_per_range

def generate_data_and_pvalues():
    data = []
    pv_counter = {pv_range: 0 for pv_range in pv_ranges}
    while not all(count >= cases_per_range for count in pv_counter.values()):
        highran = np.random.randint(lowran + 1, highestran)
        ao, bo, co, do = [int(x) for x in np.random.randint(lowran, highran, 4)]
        if len(set([ao, bo, co, do])) < 4:
            continue
        _, pv = fisher_exact([[ao, bo], [co, do]])
        for pv_min, pv_max in pv_ranges:
            if pv_min <= pv <= pv_max:
                if pv_counter[(pv_min, pv_max)] < cases_per_range:
                    smallest = min(ao, bo, co, do)
                    max_iter = 100
                    count = 0
                    pv1=0
                    fi=1
                    while pv1 < toppv and count < 100:
                        if smallest == ao:
                            a1, b1, c1, d1 = ao+fi, bo-fi, co, do
                        elif smallest == bo:
                            a1, b1, c1, d1 = ao-fi, bo+fi, co, do
                        elif smallest == co:
                            a1, b1, c1, d1 = ao, bo, co+fi, do-fi
                        elif smallest == do:
                            a1, b1, c1, d1 = ao, bo, co-fi, do+fi
                        if any(val <= 0 for val in [a1, b1, c1, d1]):
                            count +=1
                            continue
                        _, pv1 = fisher_exact([[a1, b1], [c1, d1]])
                        if pv1 < toppv:
                            fi += 1
                            count += 1
                    count = 1
                    if fi > 99:
                      continue
                    if pv1 < lowpv:
                      continue
                    ufi=1
                    pv2=0
                    while pv2 < toppv and count < 100:
                        if smallest in (ao, do):
                          a2, b2, c2, d2 = ao+ufi, bo-ufi, co-ufi, do+ufi
                        elif smallest in (bo, co):
                          a2, b2, c2, d2 = ao-ufi, bo+ufi, co+ufi, do-ufi
                        if any(val <= 0 for val in [a2, b2, c2, d2]):
                          count +=1
                          continue
                        _, pv2 = fisher_exact([[a2, b2], [c2, d2]])
                        if pv2 < toppv:
                            ufi += 1
                        count += 1
                    total = ao + bo + co + do
                    FQ1 = fi / total
                    FQ2 = ufi / total
                    RRI = abs((bo * co - ao * do) / total)
                    pRRI = RRI / total
                    size = total
                    ppv1 = ao / (ao + bo)
                    ppv2 = co / (co + do)
                    ao2, bo2, co2, do2 = ao, bo, co, do
                    if ppv1>ppv2:
                      ppv3a=(ao2-RRI)/(ao2+bo2)
                      ppv3b=(co2+RRI)/(co2+do2)
                    elif ppv1<ppv2:
                      ppv3a=(ao2+RRI)/(ao2+bo2)
                      ppv3b=(co2-RRI)/(co2+do2)
                    data.append([ao, bo, co, do, pv, a1, b1, c1, d1, pv1, fi, FQ1, a2, b2, c2, d2, pv2, ufi, FQ2, ppv1, ppv2, ppv3a, ppv3b, RRI, pRRI, size])
                    pv_counter[(pv_min, pv_max)] += 1

    columns = ['ao', 'bo', 'co', 'do', 'pv', 'a1', 'b1', 'c1', 'd1', 'pv1', 'fi', 'FQ1', 'a2', 'b2', 'c2', 'd2', 'pv2', 'ufi', 'FQ2', 'ppv1', 'ppv2', 'ppv3a', 'ppv3b', 'RRI', 'pRRI', 'size']
    df = pd.DataFrame(data, columns=columns)
    return df

df = generate_data_and_pvalues()
# OPTIONAL DOWNLOAD OF OUTPUT
#df.to_csv('Significantx.csv', index=False)
#from google.colab import files
#files.download('Significantx.csv')